In [74]:
import requests
from bs4 import BeautifulSoup


urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=1897077&sort=td&page='
urlend = '&results=items'
url = []
urls = []
pgnum = 1
while pgnum < 10:
    url.append(urlstart + str(pgnum) + urlend)
    pgnum += 1

for j in range(0, len(url)):
    r = requests.get(url[j])
    soup = BeautifulSoup(r.content)
    urltemp = str(soup)
    i = 0
    
    while i < len(urltemp):
        if urltemp[i:i+5] == '"url"':
            tempurl = urltemp[i+7:i+200].partition('"')[0]
            if 'listing' in tempurl and 'parts' not in tempurl and 'hardtop' not in tempurl and 'tools' not in tempurl and 'memorabilia' not in tempurl and 'tool kit' not in tempurl:
                if '\\' in tempurl:
                    tempurl = tempurl.replace('\\', '')
                    if tempurl not in urls:
                        urls.append(tempurl.replace('\\', ''))
        i += 1



In [75]:
print(len(urls))

83
